# COMBINED DETECTION( CAR PART MODEL AND DAMAGE MODEL)

1. Get bounding box of all damage model results
2. Use CV2 to calculate the center of all the boxes
3. Get bounding box from car part model 
4. Check if center(damage model) is in bounding box(car part model)
5. Derive information


**Inspection Results Info**

dictionary = { part : severity }

## DEPLOY CAR PART MODEL

In [1]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
carpart_artifact_dir="s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-maskrcnn-gpu-car-parts-2021-09-10-08-25-08/output/model.tar.gz"

In [3]:
carpart_estimator = TensorFlowModel(model_data= carpart_artifact_dir,
                            role=role, 
                            framework_version="1.15.3",
                            entry_point = 'inference.py',
                            source_dir="code",
                            env = {'PREDICT_USING_GRPC' : 'false'})

In [4]:
carpart_predictor = carpart_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [5]:
carpart_predictor.endpoint_name

'tensorflow-inference-2021-09-27-17-23-13-278'

In [7]:
import os
os.listdir("Damage_detection/code")

['.ipynb_checkpoints',
 'saved_model_preprocess.py',
 'requirements.txt',
 'saved_model_utils.py',
 'visualizer.py',
 'saved_model_config.py',
 'saved_model_inference.py',
 'config.py',
 'inference.py']

## DEPLOY DAMAGE MODEL

In [24]:
# cardamage_artifact_dir = "s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-gpu-car-damage-detection-2021-09-09-13-51-00/output/model.tar.gz"
cardamage_artifact_dir ="s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-gpu-car-damage-detection-2021-09-27-11-27-50/output/model.tar.gz"


In [25]:
cardamage_estimator = TensorFlowModel(model_data= cardamage_artifact_dir,
                            role=role, 
                            framework_version="1.15.3",
                            entry_point = 'inference.py',
                            source_dir="Damage_detection/code",
                            env = {'PREDICT_USING_GRPC' : 'false'})

In [26]:
cardamage_predictor = cardamage_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------!

In [27]:
cardamage_predictor.endpoint_name

'tensorflow-inference-2021-09-27-17-50-52-855'

## INVOKE

In [28]:
import visualizer as visualize
import skimage
import os
import numpy as np
import boto3
import json
import numpy as np
from PIL import Image
from IPython.display import display
import io
import base64
from io import BytesIO


In [29]:
def client_predictor(predictor,image):  
    endpoint_name = predictor.endpoint_name
    runtime_client = boto3.client('runtime.sagemaker')
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType='application/x-image',     
                                           Body=image)
    res = response['Body'].read().decode('ascii')
    r = json.loads(res)

    
    return r

In [14]:
def compute_results(file):
    car_image = open(file, 'rb').read()
    #part detection
    part_detection = client_predictor(carpart_predictor, car_image)

    #damage detection
    damage_detection = client_predictor(cardamage_predictor, car_image)

    return part_detection,damage_detection

## PERFORM COMPARE RESULTS AND EXTRACT INFO

In [15]:
#Bounding Box Centroid
#center = ((x1+x2)/2, (y1+y2)/2)
def extract_centroid(bbox):
    x1 = bbox[0]
    y1 = bbox[1]
    x2 = bbox[2]
    y2 = bbox[3]
    
    centroid = ((x1+x2)/2, (y1+y2)/2)
    
    return centroid

In [16]:
#to judge a point(x0,y0) is in the rectangle, just to check if a < x0 < a+c and b < y0 < b + d

def rectContains(rect,pt):
    logic = rect[0] < pt[0] < rect[0]+rect[2] and rect[1] < pt[1] < rect[1]+rect[3]
    return logic



In [17]:
def extract_info(payload):
    bbox = payload['rois']
    class_ids = payload['class']
    scores = payload['scores']
    
    return bbox,class_ids
    

In [18]:
def isInRect(target_rect,child_rect):
    centroid = extract_centroid(child_rect)
    return rectContains(target_rect,centroid)

## PREDICTION

In [19]:
def dict_result(part_rois, damage_rois, part_ids,damage_ids):
    
    damage_cat = np.array([0,'none','low', 'medium', 'high'])
    part_cat = np.array([0,
               "hood", 
               "front-bumper", 
               "rear-bumper", 
               "front-wind-screen",
               "rear-windscreen",
               "left-mirror", 
               "right-mirror", 
               "front-left-fender", 
               "font-right-fender", 
               "left-tail-light", 
               "right-tail-light", 
               "trunk", 
               "front-left-door", 
               "front-right-door", 
               "back-left-door", 
               "back-right-door", 
               "roof", 
               "tire", 
               "left-headlight", 
               "right-headlight", 
               "gas-tank", 
               "rear-left-fender", 
               "rear-right-fender", 
               "window"])
    dict_results = {}

    for i, part_rect in enumerate(part_rois):
        for j, damage_rect in enumerate(damage_rois):
            if isInRect(part_rect,damage_rect):
                dict_results[ part_cat[part_ids[i]] ] = damage_cat[damage_ids[j]]
                
    return dict_results

In [20]:
list_files = os.listdir("sample_files")

In [21]:
list_files

['car wreck_-ef43703c49fb57a010fa9ef1c6c28dbeffd491793e7333bb4299fe4490d95a7f.jpg',
 'car wreck_-2432b535d4b3f5c96f4090d7aa69d76ba64577f4f85c9bb55817af2dd7c65aec.jpg',
 'car wreck_-b5a3c6729b65b9d02d1398e4f46d17ed2cd1f5f26cc97cf720fc6016ca1208a4.jpg',
 '9.jpg',
 'car wreck_-af76c4700ff3dc62083b2b54eb2d107a5c90526f56bcf072f31b662ca15c33ab.jpg',
 'car wreck_-b0a3ab05fed79d0a3e134c71179c1834768e2d2e2f1d402f0cbdae715354e0f2.jpg',
 'car crash_-f44e3172f0c99d67cac362ea118af93fe6a8868c3d6ed189d8983add49c90c13.jpg',
 'car wreck_-16578778651c5856199faf5a3419a63ac033b2073026a47e2491b9c6f5117cb9.jpg',
 'cardamaged-df735878beb4b81768983862a31d4ffd146d7b9512466f74d50dd112bdd35b79.jpg',
 'car wreck_-c9334facae81b43badac6379cc48f73d8420db1f5eb81c736a01a4fb0d7bae0f.jpg',
 '10.jpg']

In [30]:
for file in list_files:
    image_path = f"sample_files/{file}"
    p_r, d_r = compute_results(image_path)
    
    viz_img = d_r['image']
    decoded = base64.b64decode(viz_img)
    image = io.BytesIO(decoded)
    img = Image.open(image)
    display(img)
    
    d_rois, d_class_ids = extract_info(d_r['data'])
    p_rois, p_class_ids = extract_info(p_r)
    result = dict_result(p_rois,d_rois,p_class_ids, d_class_ids) 
    print(result)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0</center>
</body>
</html>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2021-09-27-17-50-52-855 in account 186402456289 for more information.

## CLEAN UP

In [33]:
# carpart_predictor.delete_endpoint()
# cardamage_predictor.delete_endpoint()